In [22]:
import torch
from neural_diff_eq.utils import jac, laplacian

Re = 10
def pde(u, input):
    jac_t = jac(u, input['t'])
    jac_x = jac(u, input['x'])
    conv = torch.bmm(jac_x, u.reshape(u.shape[0], 2, 1))
    l_1 = laplacian(u[:, 0], input['x'], grad=jac_x[:, 0, :])
    l_2 = laplacian(u[:, 1], input['x'], grad=jac_x[:, 1, :])
    laplace_vec = torch.cat((l_1, l_2), dim=1)
    return (jac_t + conv).reshape(u.shape[0], 2) - 1/Re * laplace_vec


In [4]:
import torch
from neural_diff_eq.utils import grad, jac, laplacian
def f(input):
    out = torch.zeros((input.shape[0], 2))
    out[:, :1] = input[:, :1]**2*input[:, 1:2]**2
    out[:, 1:] = torch.sin(input[:, 2:]*input[:, 1:2])
    return out
x = torch.tensor([[1.0, 2], [1, 1], [3, 4], [3, 0]], requires_grad=True)
t = torch.tensor([[3.0], [0], [1], [2]], requires_grad=True)
inp = torch.cat((x, t), dim=1)
o = f(inp)
print(o)

tensor([[  4.0000,  -0.2794],
        [  1.0000,   0.0000],
        [144.0000,  -0.7568],
        [  0.0000,   0.0000]], grad_fn=<CopySlices>)


In [5]:
jac_t = jac(o, t)
jac_x = jac(o, x)
print(jac_t)
print(jac_x)

tensor([[[ 0.0000],
         [ 1.9203]],

        [[ 0.0000],
         [ 1.0000]],

        [[ 0.0000],
         [-2.6146]],

        [[ 0.0000],
         [ 0.0000]]], grad_fn=<StackBackward>)
tensor([[[ 8.0000,  4.0000],
         [ 0.0000,  2.8805]],

        [[ 2.0000,  2.0000],
         [ 0.0000,  0.0000]],

        [[96.0000, 72.0000],
         [ 0.0000, -0.6536]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  2.0000]]], grad_fn=<StackBackward>)


In [12]:
conv = torch.bmm(jac_x, o.reshape(o.shape[0], 2, 1))
print(conv)
(jac_t + conv).reshape(o.shape[0], 2)

tensor([[[ 3.0882e+01],
         [-8.0486e-01]],

        [[ 2.0000e+00],
         [ 0.0000e+00]],

        [[ 1.3770e+04],
         [ 4.9468e-01]],

        [[ 0.0000e+00],
         [ 0.0000e+00]]], grad_fn=<BmmBackward0>)


tensor([[ 3.0882e+01,  1.1155e+00],
        [ 2.0000e+00,  1.0000e+00],
        [ 1.3770e+04, -2.1199e+00],
        [ 0.0000e+00,  0.0000e+00]], grad_fn=<ViewBackward>)

In [17]:
l_1 = laplacian(o[:, 0], x, grad=jac_x[:, 0, :])
print(laplacian(o[:, 0], x))
print(l_1)

tensor([[10.],
        [ 4.],
        [50.],
        [18.]], grad_fn=<AddBackward0>)
tensor([[10.],
        [ 4.],
        [50.],
        [18.]], grad_fn=<AddBackward0>)


In [18]:
l_1 = laplacian(o[:, 1], x, grad=jac_x[:, 1, :])
print(laplacian(o[:, 1], x))
print(l_1)

tensor([[2.5147],
        [0.0000],
        [0.7568],
        [0.0000]], grad_fn=<AddBackward0>)
tensor([[2.5147],
        [0.0000],
        [0.7568],
        [0.0000]], grad_fn=<AddBackward0>)


In [21]:
(jac_t + conv).reshape(o.shape[0], 2) - 1/5 * torch.cat((l_1, l_2), dim=1)

tensor([[ 3.0379e+01,  6.1253e-01],
        [ 2.0000e+00,  1.0000e+00],
        [ 1.3769e+04, -2.2713e+00],
        [ 0.0000e+00,  0.0000e+00]], grad_fn=<SubBackward0>)